In [ ]:
import json
import os
import numpy as np
import evaluate
from sentence_transformers import SentenceTransformer, util
import torch

2025-05-15 11:35:44.584197: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Applications/anaconda3/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/Applications/anac

In [45]:
def read_jsonl(filename):
    """
    Read a jsonlines file into a list of dictionaries.

    Args:
        filename (str): Path to input file.

    Returns:
        ([dict]): List of dictionaries.
    """
    with open(filename, "r") as f:
        lines = f.readlines()
        samples = []
        for l in lines:
            samples.append(json.loads(l))
        return samples


def dump_jsonl(d, filename):
    """
    Dump a list of dictionaries to file as jsonlines.

    Args:
        d ([dict]): List of dictionaries.
        filename (str): Path to output file.

    Returns:
        None
    """
    with open(filename, "w") as f:
        for entry in d:
            f.write(json.dumps(entry) + "\n")

In [ ]:
#result object
class AssertInfo(object):
    def __init__(self, idx, res_idx, content_keys):
        self.val_index = idx
        self.res_index = res_idx
        self.data = []
        self.structure_id = True
        self.test_account = {e: set() for e in content_keys}
        self.gold_account = {e: set() for e in content_keys}

In [ ]:
metric = evaluate.load("/Users/yuz/Work/reprod/evaluate/metrics/rouge/rouge.py")

def get_tokens(s):
    if isinstance(s, str):
        stripped =  [e.strip() for e in s.split(" ")]
    else:
        stripped = [s]
    return [e for e in stripped if e]


"""
    {
        "Sample": [{
            "Diseases": "Emphysematous cystitis", 
            "Symptoms": ["Nausea", "Vomiting", "Macrohematuria"], 
            "TherapeuticsInformation": [{
                "TherapeuticsApproach": "Antibiotics", 
                "CureRate": "0", 
                "DrugFrequency": ""}, {
                "TherapeuticsApproach": "Simple cystectomy and ureterocutaneostomy", 
                "CureRate": "1", 
                "DrugFrequency": ""
            }], 
        "DiagnosisMethods": ["Cystoscopy", "Histological examination",
                             "Urine and mucosal surface cultures"], 
        "Causes": "Escherichia coli infection", 
        "MedicalHistory": "Insulin-dependent diabetes mellitus (IDDM)"
        }]
    }
"""

def diff_json(response_data, assert_data, content_keys, assert_info, content_key):
    if isinstance(response_data, dict):
        for key in assert_data:
            if key not in response_data:
                print(f'structure unidentified with id {assert_info.val_index} in dict condition [1] with content key: {content_key} and response_data: {response_data}')
                assert_info.structure_id = False
                return
            
        for key in response_data:
            if key in assert_data:
                diff_json(response_data[key], assert_data[key], content_keys, assert_info, key)
            else:
                print(f'structure unidentified with id {assert_info.val_index} in dict condition [2] with content key: {content_key} and response_data: {response_data}')
                assert_info.structure_id = False
                return
    elif isinstance(response_data, list):
        if len(response_data) != 0 and len(assert_data)!=0 and len(response_data) != len(assert_data) and isinstance(assert_data[0], dict):
            print(f'structure unidentified with id {assert_info.val_index} in list condition [1] with content key: {content_key} and response_data: {response_data}')
            assert_info.structure_id = False
            return

        if response_data and assert_data and not isinstance(response_data[0], type(assert_data[0])):
            print(f'structure unidentified with id {assert_info.val_index} in list condition [2] with content key: {content_key} and response_data: {response_data}')
            assert_info.structure_id = False
            return 

        if content_key in content_keys:
            len_response_data = 0
            len_assert_data = 0
            if response_data and not isinstance(response_data[0], dict):
                len_response_data = len(response_data)
                for res_str in response_data:
                    assert_info.test_account[content_key].add(str(res_str).strip().lower())
            if assert_data and not isinstance(assert_data[0], dict):
                len_assert_data = len(assert_data)
                for res_str in assert_data:
                    assert_info.gold_account[content_key].add(str(res_str).strip().lower())
            if len_assert_data != len_response_data:
                pass
                # print(f'values length not equality with content key {content_key} and index {assert_info.val_index}')
            
        if response_data:
            if isinstance(response_data[0], dict):
                response_data = sorted(response_data, key=lambda x: x[list(response_data[0].keys())[0]])
            else:
                response_data = sorted(response_data)
        if assert_data:
            if isinstance(assert_data[0], dict):
                assert_data = sorted(assert_data, key=lambda x: x[list(assert_data[0].keys())[0]])
            else:
                assert_data = sorted(assert_data)
        
        
        if response_data and assert_data and isinstance(response_data[0], dict):
            if content_key in content_keys:
                print(f'content key: {content_key} except with dict type object!!!')
            for src_list, dst_list in zip(response_data, assert_data):
                diff_json(src_list, dst_list, content_keys, assert_info, content_key)
    else:
        if content_key in content_keys:
            if response_data:
                assert_info.test_account[content_key].add(str(response_data).strip().lower())

            if assert_data:
                assert_info.gold_account[content_key].add(str(assert_data).strip().lower())
    pass

In [48]:
Content_keys = ['Diseases', 'Symptoms', 
                'DiagnosisMethods', 'Causes', 'MedicalHistory',
                'TherapeuticsApproach', 'CureRate', 'DrugFrequency']

In [ ]:
#truth results

trigger = 'treatment'
treatment_vals = []
val_results = []
for fold in range(5):
    val_path = f'./chatbot_checked_{trigger}/fold{fold}/val_{trigger}_fold{fold}.jsonl'
    current_val_result = read_jsonl(val_path)
    val_results.extend(current_val_result)
    for item in current_val_result:
        treatment_vals.append(json.loads(item['messages'][2]['content']))

trigger = 'cured'
cured_vals = []
val_results = []
for fold in range(5):
    val_path = f'./chatbot_checked_{trigger}/fold{fold}/val_{trigger}_fold{fold}.jsonl'
    current_val_result = read_jsonl(val_path)
    val_results.extend(current_val_result)
    for item in current_val_result:
        cured_vals.append(json.loads(item['messages'][2]['content']))

In [ ]:
##openai cured
openai_cured_results = []
openai_cured_val_index = []
run = []
idx = -1
for fold in range(5):
    result_file_name = f"./chatbot_checked_cured/fold{fold}/batch_results_cured_fold{fold}.jsonl"
    with open(result_file_name, 'r') as file:
        for line in file:
            # Parsing the JSON string into a dict and appending to the list of results
            idx += 1
            json_object = json.loads(line.strip())
            openai_cured_results.append(json.loads(json_object['response']['body']['choices'][0]['message']['content']))
            run.append(json_object['response']['body']['choices'][0]['message']['content'])
            openai_cured_val_index.append(idx)

In [ ]:
##openai treatment
openai_treatment_results = []
openai_treatment_val_index = []
run = []
idx = -1
for fold in range(5):
    result_file_name = f"./chatbot_checked_treatment/fold{fold}/batch_results_treatment_fold{fold}.jsonl"
    with open(result_file_name, 'r') as file:
        for line in file:
            # Parsing the JSON string into a dict and appending to the list of results
            idx += 1
            json_object = json.loads(line.strip())
            openai_treatment_results.append(json.loads(json_object['response']['body']['choices'][0]['message']['content']))
            run.append(json_object['response']['body']['choices'][0]['message']['content'])
            openai_treatment_val_index.append(idx)

In [ ]:
##llama cured
llama_cured_results = []
llama_cured_val_index = []
idx = -1
cured_run = []
for fold in range(5):
    current_run = read_jsonl(os.path.join(f'./results_cured_checked_chat', f'run_cured_{fold}.jsonl'))
    cured_run.extend(current_run)
    for result in current_run:
        idx += 1
        try:
            completion = json.loads(result['completion'])
        except:
            print(result['completion'])
            continue
        
        llama_cured_val_index.append(idx)
        llama_cured_results.append(completion)

{"Sample": [{"Diseases": "Secondary hyperaldosteronism", "Symptoms": [], "TherapeuticsInformation": [{"TherapeuticsApproach": "Atypical antipsychotic", "CureRate": "1", "DrugFrequency": ""}, {"TherapeuticsApproach": "Atypical antipsychotic", "CureRate": "1", "DrugFrequency": ""}, {"TherapeuticsApproach": "Atypical antipsychotic", "CureRate": "1", "DrugFrequency": ""}, {"TherapeuticsApproach": "Atypical antipsychotic", "CureRate": "1", "DrugFrequency": ""}, {"TherapeuticsApproach": "Atypical antipsychotic", "CureRate": "1", "DrugFrequency": ""}, {"TherapeuticsApproach": "Atypical antipsychotic", "CureRate": "1", "DrugFrequency": ""}, {"TherapeuticsApproach": "Atypical antipsychotic", "CureRate": "1", "DrugFrequency": ""}, {"TherapeuticsApproach": "Atypical antipsychotic", "CureRate": "1", "DrugFrequency": ""}, {"TherapeuticsApproach": "Atypical antipsychotic", "CureRate": "1", "DrugFrequency": ""}, {"TherapeuticsApproach": "Atypical antipsychotic", "CureRate": "1", "DrugFrequency": ""},

In [ ]:
##llama treatment
llama_treatment_results = []
llama_treatment_val_index = []
idx = -1
treatment_run = []
for fold in range(5):
    current_run = read_jsonl(os.path.join(f'./results_treatment_checked_chat', f'run_treatment_{fold}.jsonl'))
    treatment_run.extend(current_run)
    for result in current_run:
        idx += 1
        try:
            completion = json.loads(result['completion'])
        except:
            print(result['completion'])
            continue
        
        llama_treatment_val_index.append(idx)
        llama_treatment_results.append(completion)

{"Sample": [{"Diseases": "cancer of the adrenal cortex", "Symptoms": [], "TherapeuticsInformation": [{"TherapeuticsApproach": "O,P"-DDD, "CureRate": "", "DrugFrequency": ""}], "DiagnosisMethods": [], "Causes": "", "MedicalHistory": ""}]}




# evaluate

In [60]:
def soft_match(query, value, sim_model):
    device = torch.device("cpu")
    embedding_1 = sim_model.encode(query, convert_to_tensor=True, device=device) # encode on CPU
    embedding_2 = sim_model.encode(value, convert_to_tensor=True, device=device) # encode on CPU
    sim_matrix = util.pytorch_cos_sim(embedding_1, embedding_2)
    return sim_matrix

In [ ]:
#cured
openai_cured_assertInfos = []
llama_cured_assertInfos = []
res_idx = 0
openai_cured_gold = []
openai_cured_pred = []
llama_cured_gold = []
llama_cured_pred = []
for idx in range(len(cured_vals)):
    if idx in openai_cured_val_index:
        openai_cured_gold.append(val_results[idx]['messages'][2]['content'])
        openai_cured_pred.append(run[idx])
        openai_cured_assertInfos.append(AssertInfo(idx, res_idx, Content_keys))
        diff_json(openai_cured_results[res_idx], cured_vals[idx], Content_keys, openai_cured_assertInfos[res_idx], 'root')
        res_idx += 1
        
res_idx = 0
for idx in range(len(cured_vals)):
    if idx in llama_cured_val_index:
        llama_cured_gold.append(val_results[idx]['messages'][2]['content'])
        llama_cured_pred.append(run[idx])
        llama_cured_assertInfos.append(AssertInfo(idx, res_idx, Content_keys))
        diff_json(openai_cured_results[res_idx], cured_vals[idx], Content_keys, llama_cured_assertInfos[res_idx], 'root')
        res_idx += 1

In [ ]:
#treatment
openai_treatment_assertInfos = []
llama_treatment_assertInfos = []
res_idx = 0
openai_treatment_gold = []
openai_treatment_pred = []
llama_treatment_gold = []
llama_treatment_pred = []
for idx in range(len(treatment_vals)):
    if idx in openai_treatment_val_index:
        openai_treatment_gold.append(val_results[idx]['messages'][2]['content'])
        openai_treatment_pred.append(run[idx]['completion'].replace('\\\"',""))
        openai_treatment_assertInfos.append(AssertInfo(idx, res_idx, Content_keys))
        diff_json(openai_treatment_results[res_idx], treatment_vals[idx], Content_keys, openai_treatment_assertInfos[res_idx], 'root')
        res_idx += 1

res_idx = 0
for idx in range(len(treatment_vals)):
    if idx in llama_treatment_val_index:
        llama_treatment_gold.append(val_results[idx]['messages'][2]['content'])
        llama_treatment_pred.append(run[idx]['completion'].replace('\\\"',""))
        llama_treatment_assertInfos.append(AssertInfo(idx, res_idx, Content_keys))
        diff_json(llama_treatment_assertInfos[res_idx], treatment_vals[idx], Content_keys, llama_treatment_assertInfos[res_idx], 'root')
        res_idx += 1
        

In [ ]:
for assertInfos in [openai_cured_assertInfos, openai_treatment_assertInfos, llama_cured_assertInfos, llama_treatment_assertInfos]:
    assertlength = len(assertInfos)
    structure_negative = 0
    for assertItem in assertInfos:
        if assertItem.structure_id == False:
            structure_negative += 1
    structure_postive = assertlength - structure_negative
    print(structure_postive / len(cured_vals))

In [ ]:
device = torch.device("cpu")
sim_model = SentenceTransformer('/Users/yuz/Work/reprod/sentence-transformers')
sim_model = sim_model.to(device)
for gold, pred in [(openai_cured_gold, openai_cured_pred), (openai_treatment_gold, openai_treatment_pred), 
                   (llama_cured_gold, llama_cured_pred), (llama_treatment_gold, llama_treatment_pred)]:
    gold_n, pred_n, total_score = 0, 0, 0
    for gold_item, pred_item in zip(gold, pred):
        gold_n += len(gold_item)
        pred_n += len(pred_item)
        scores = soft_match(gold_item, pred_item, sim_model)[0][0]
        total_score += scores
        # break
    print(total_score / len(gold))

In [ ]:
scores_dict = {}
for name, assertInfos in [("openai_cured", openai_cured_assertInfos), 
                    ("openai_treat", openai_treatment_assertInfos), 
                    ("llama_cured", llama_cured_assertInfos), 
                    ("llama_treat", llama_treatment_assertInfos)]:
    overall_p_scores_rouge = {e: [] for e in Content_keys}
    overall_r_scores_rouge = {e: [] for e in Content_keys}
    overall_f1_scores_rouge = {e: [] for e in Content_keys}
    overall_scores_rouge = {e: [] for e in Content_keys}

    overall_p_scores_em = {e: [] for e in Content_keys}
    overall_r_scores_em = {e: [] for e in Content_keys}
    overall_f1_scores_em = {e: [] for e in Content_keys}
    overall_scores_em = {e: [] for e in Content_keys}
    for idx in range(len(assertInfos)):
        if not assertInfos[idx].structure_id:
            continue
        for key in Content_keys:
            test_account = list(assertInfos[idx].test_account[key])
            gold_account = list(assertInfos[idx].gold_account[key])
            if (len(test_account) == 0 and len(gold_account) > 0) or (len(test_account) > 0 and len(gold_account) == 0):
                overall_p_scores_rouge[key].append(0)
                overall_r_scores_rouge[key].append(0)
                overall_f1_scores_rouge[key].append(0)
                overall_p_scores_em[key].append(0)
                overall_r_scores_em[key].append(0)
                overall_f1_scores_em[key].append(0)
            elif len(test_account) == 0 and len(gold_account) == 0:
                overall_p_scores_rouge[key].append(1.0)
                overall_r_scores_rouge[key].append(1.0)
                overall_f1_scores_rouge[key].append(1.0)
                overall_p_scores_em[key].append(1.0)
                overall_r_scores_em[key].append(1.0)
                overall_f1_scores_em[key].append(1.0)
            else:
                total_score = 0
                em_score = 0
                if len(test_account) < len(gold_account):
                    for gold_item in gold_account:
                        max_item_score = np.float64(0)
                        for test_item in test_account:
                            if test_item == gold_item:
                                em_score += 1
                            current_score = metric.compute(predictions = [test_item], references = [gold_item])['rougeLsum']
                            if current_score > max_item_score:
                                max_item_score = current_score
                        total_score += max_item_score
                else:
                    for test_item in test_account:
                        max_item_score = np.float64(0)
                        for gold_item in gold_account:
                            if gold_item == test_item:
                                em_score += 1
                            current_score = metric.compute(predictions = [test_item], references = [gold_item])['rougeLsum']
                            if current_score > max_item_score:
                                max_item_score = current_score
                        total_score += max_item_score
                # print(f'key :{key} score: {total_score} len: {len(test_account)}')
                p_em_score = em_score / len(test_account)
                r_em_score = em_score / len(gold_account)
                overall_p_scores_em[key].append(p_em_score)
                overall_r_scores_em[key].append(r_em_score)
                if p_em_score == 0 and r_em_score == 0:
                    overall_f1_scores_em[key].append(0)
                else:
                    f1 = 2 * ( p_em_score * r_em_score) / (p_em_score + r_em_score)
                    overall_f1_scores_em[key].append(f1)
                p_score = total_score / len(test_account)
                r_score = total_score / len(gold_account)
                overall_p_scores_rouge[key].append(p_score)
                overall_r_scores_rouge[key].append(r_score)
                if p_score == 0 and r_score == 0:
                    overall_f1_scores_rouge[key].append(0)
                else:
                    f1 = 2 * ( p_score * r_score) / (p_score + r_score)
                    overall_f1_scores_rouge[key].append(f1)
                    
    scores_dict[name] = {}
    scores_dict[name]["overall_p_scores_rouge"] = overall_p_scores_rouge
    scores_dict[name]["overall_r_scores_rouge"] = overall_r_scores_rouge
    scores_dict[name]["overall_f1_scores_rouge"] = overall_f1_scores_rouge
    scores_dict[name]["overall_scores_rouge"] = overall_scores_rouge
    
    scores_dict[name]["overall_p_scores_em"] = overall_p_scores_em
    scores_dict[name]["overall_r_scores_em"] = overall_r_scores_em
    scores_dict[name]["overall_f1_scores_em"] = overall_f1_scores_em
    scores_dict[name]["overall_scores_em"] = overall_scores_em

In [ ]:
for name in ["openai_cured", "openai_treat", "llama_cured", "llama_treat"]:
    avg_p_scores_em, avg_r_scores_em, avg_f1_scores_em, avg_p_scores_rl, avg_r_scores_rl, avg_f1_scores_rl = 0, 0, 0, 0, 0, 0
    for key in Content_keys:
        avg_p_scores_em += np.average(scores_dict[name]["overall_p_scores_em"][key])
        avg_r_scores_em += np.average(scores_dict[name]["overall_r_scores_em"][key])
        avg_f1_scores_em += np.average(scores_dict[name]["overall_f1_scores_em"][key])
        avg_p_scores_rl += np.average(scores_dict[name]["overall_p_scores_rouge"][key])
        avg_r_scores_rl += np.average(scores_dict[name]["overall_r_scores_rouge"][key])
        avg_f1_scores_rl += np.average(scores_dict[name]["overall_f1_scores_rouge"][key])
    print(f'{name} avg_p_scores_em: {avg_p_scores_em / len(Content_keys)}')
    print(f'{name} avg_r_scores_em: {avg_r_scores_em / len(Content_keys)}')
    print(f'{name} avg_f1_scores_em: {avg_f1_scores_em / len(Content_keys)}')
    print(f'{name} avg_p_scores_rouge: {avg_p_scores_rl / len(Content_keys)}')
    print(f'{name} avg_r_scores_rouge: {avg_r_scores_rl / len(Content_keys)}')
    print(f'{name} avg_f1_scores_rouge: {avg_f1_scores_rl / len(Content_keys)}')